## 🎯 Pregunta de negocio  
**¿Cómo puede Mercado Libre identificar a los sellers más relevantes y segmentarlos para diseñar estrategias comerciales personalizadas?**

🧠 **Hipótesis inicial:**  
Los sellers con **alto stock**, **muchas publicaciones** y **cero productos reacondicionados** tienden a ser más relevantes para el negocio.

In [5]:
import sys
from IPython.display import Markdown, display
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
sys.path.append("C:/Users/dario/Documents/worksapce/sellers_analitycs_meli")

from meli_insight_engine.llm.prompts.templates import BASIC_RECOMMENDATION_PROMPT_JSON, BASIC_RECOMMENDATION_PROMPT_HIPOTESIS
from meli_insight_engine.llm.agents.agent_template import TemplateAgent
from meli_insight_engine.core import (
    conectar_duckdb,
    registrar_csvs_como_vistas,
    verificar_vistas,
    analisis_inicial_completo,
    guardar_resultados_como_txt
)



# 🧠 Descubrimiento inicial del dataset con MAD-G

Este notebook realiza una **revisión estructural y exploración inicial** del archivo `df_challenge.csv`, en el contexto del challenge de segmentación de sellers de Mercado Libre.

📍 **Objetivo general:**
Comprender la composición y calidad del dataset para diseñar una solución de clusterización efectiva y alineada con necesidades comerciales.

📌 **Lo que haremos en esta etapa:**
- Analizar la estructura del dataset (columnas, tipos de datos, duplicados, nulos, constantes).
- Detectar variables relevantes para caracterizar a los sellers.
- Evaluar posibles problemas de calidad de datos que deban corregirse o imputarse.
- Generar **hipótesis de negocio iniciales** con ayuda de un modelo de lenguaje (LLM), a partir de la observación de los datos.
  
🧩 **¿Qué estamos construyendo?**
Una herramienta analítica que combina la exploración de datos tradicional con un asistente de IA (LLM), capaz de:
- Proponer agrupaciones lógicas de vendedores.
- Sugerir estrategias de segmentación.
- Enriquecer el entendimiento de los datos con razonamiento automatizado.

Esta exploración será la base para la ingeniería de features y el modelado posterior.


## ⚙️ Configuración del entorno y carga de datos

### Analisis de datos para levantar hipotesis usando LLM 

In [ ]:

# Crear conexión a DuckDB
con = conectar_duckdb()

# Ruta a tu carpeta con archivos .csv de prueba (asegúrate de que exista y tenga al menos un .csv)
folder_path = "../data/"
# Registrar vistas
registrar_csvs_como_vistas(con, folder_path)


In [121]:


# Verificar vistas
print("✅ Vistas registradas:")
print(verificar_vistas(con))

# Reemplaza con el nombre real del archivo sin .csv y sin guiones (los guiones se vuelven guiones bajos)
nombre_tabla = "data.df_challenge_meli"# Ejemplo: si el archivo se llama "ventas-julio.csv", usa "data.ventas_julio"

resultados = analisis_inicial_completo(con, "data.df_challenge_meli")
guardar_resultados_como_txt(resultados)


✅ Vistas registradas:
                          table_name
0                data.categorias_mco
1             data.df_challenge_meli
2      data.df_challenge_meli_limpio
3     data.featured_df_challeng_meli
4  data.featured_df_clustered_dbscan


C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:358: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = con.execute(f'SELECT "{col}" FROM "{tabla}" WHERE "{col}" IS NOT NULL LIMIT 100').fetchdf()
C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:358: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = con.execute(f'SELECT "{col}" FROM "{tabla}" WHERE "{col}" IS NOT NULL LIMIT 100').fetchdf()
C:\Users/dario/Documents/worksapce/sellers_analitycs_meli\meli_insight_engine\core\inspector.py:358: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is 

✅ Resultados guardados en: ../data/outputs_prompts/inspector_stats.txt


In [ ]:

# Ruta del archivo .txt
ruta_txt = "../data/outputs_prompts/inspector_stats.txt"

# Leer contenido del archivo
with open(ruta_txt, "r", encoding="utf-8") as f:
    texto = f.read()

    # Instanciar el agente con el template y el texto leído
    agent = TemplateAgent(prompt_template=BASIC_RECOMMENDATION_PROMPT_JSON,template_name= "BASIC_RECOMMENDATION_PROMPT_JSON" )

# Ejecutar
respuesta = agent.run(input_path=ruta_txt)

# Mostrar respuesta
print(respuesta)

../data/outputs_prompts\BASIC_RECOMMENDATION_PROMPT_JSON.txt


In [ ]:


# Ruta del archivo .txt
ruta_txt = "../data/outputs_prompts/BASIC_RECOMMENDATION_PROMPT_JSON.txt"

# Leer contenido del archivo
with open(ruta_txt, "r", encoding="utf-8") as f:
    texto = f.read()

    # Instanciar el agente con el template y el texto leído
    agent = TemplateAgent(prompt_template=BASIC_RECOMMENDATION_PROMPT_HIPOTESIS,template_name= "BASIC_RECOMMENDATION_PROMPT_HIPOTESIS" )

# Ejecutar
respuesta = agent.run(input_path=ruta_txt)

# Mostrar respuesta
print(respuesta)

c:\Users\dario\anaconda3\envs\meli_challenge\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\dario\anaconda3\envs\meli_challenge\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


../data/outputs_prompts\BASIC_RECOMMENDATION_PROMPT_HIPOTESIS.txt


#### resultados de MDA-G (models for data Analysis)

In [6]:

# Ruta al archivo externo (por ejemplo, en una carpeta docs o en el root)
ruta_readme = "../data/outputs_prompts/BASIC_RECOMMENDATION_PROMPT_HIPOTESIS.txt"

# Leer y mostrar
with open(ruta_readme, "r", encoding="utf-8") as archivo:
    contenido = archivo.read()

display(Markdown(contenido))


### **Reporte de Comprensión Inicial del Dataset**

---

#### **1. Estructura Básica**
- **Total registros**: 185,250  
- **Total variables**: 14  
- **Variables clave identificadas**:  
  - **[Numérica Continua] `price`**: Precio actual del producto (variable objetivo potencial).  
  - **[Numérica Continua] `regular_price`**: Precio regular (alta tasa de nulos, requiere imputación).  
  - **[Categórica Ordinal] `seller_reputation`**: Reputación del vendedor (jerarquía implícita útil para análisis de confianza).  
  - **[Booleana] `is_refurbished`**: Indica si el producto es reacondicionado (desequilibrio extremo, relevante para segmentación).  
  - **[Nominal] `categoria`**: Categoría del producto (posible driver de precios y stock).  

---

#### **2. Hallazgos de Calidad**  
- **`regular_price`**:  
  - **Tipo de problema**: Missing values (73.03% nulos).  
  - **Severidad**: Alta (afecta análisis de descuentos/precios).  
  - **Acción sugerida**: Imputación predictiva (modelo basado en `price` y categoría).  

- **`titulo`**:  
  - **Tipo de problema**: Alta cardinalidad (174,746 valores únicos).  
  - **Severidad**: Media (puede generar ruido en modelos).  
  - **Acción sugerida**: Hashing o NLP para extraer características clave.  

- **`is_refurbished`**:  
  - **Tipo de problema**: Desequilibrio (99.6% "False").  
  - **Severidad**: Alta (impacta modelos de clasificación).  
  - **Acción sugerida**: Rebalanceo con SMOTE o undersampling.  

- **`tim_day`**:  
  - **Tipo de problema**: Valor constante.  
  - **Severidad**: Baja (sin impacto analítico).  
  - **Acción sugerida**: Eliminar columna.  

- **`price`**:  
  - **Tipo de problema**: Outliers (valores hasta 4.7B).  
  - **Severidad**: Alta (distorsiona estadísticas).  
  - **Acción sugerida**: Clipping (límite superior: 999,999).  

---

#### **3. Relaciones Potenciales**  
- **`price` ↔ `is_refurbished`**: Diferencias de precio por condición del producto. **Método**: ANOVA.  
- **`seller_reputation` ↔ `price`**: Correlación entre reputación y precios. **Método**: Spearman (ordinal-numérica).  
- **`categoria` ↔ `stock`**: Patrones de stock por categoría. **Método**: Test Chi-cuadrado.  

---

#### **4. Recomendaciones para EDA**  
- **Grupo 1**: `price`, `regular_price`, `is_refurbished`  
  - **Objetivo**: Analizar estrategias de precios y descuentos.  
- **Grupo 2**: `seller_reputation`, `categoria`, `stock`  
  - **Objetivo**: Explorar segmentación por confianza del vendedor y disponibilidad.  

**Transformaciones previas**:  
- **`regular_price`**: Aplicar log-transform (mejorar linealidad).  
- **`seller_reputation`**: Codificar ordinalmente (ej: "newbie"=0, "green_platinum"=4).  

---

#### **5. Advertencias Clave**  
- **Limitación 1**: Alta cardinalidad en `titulo` y `url` (requiere eliminación o procesamiento avanzado).  
- **Limitación 2**: Desequilibrio en `is_refurbished` (puede sesgar modelos predictivos).  

---

#### **6. Conclusión Ejecutiva**  
- **Calidad del dataset**: Aceptable con deficiencias controlables (alta tasa de nulos en `regular_price`, outliers en `price`).  
- **Variables prometedoras**: `price` (análisis de precios), `seller_reputation` (segmentación), `categoria` (patrones de inventario).  
- **Riesgos principales**:  
  - Outliers en precios distorsionan métricas.  
  - Desequilibrio en variables booleanas afecta modelos.  
- **Valor potencial**: Dataset robusto para análisis de precios y reputación si se aplican imputaciones y transformaciones sugeridas.  

**Recomendación final**: Priorizar limpieza de outliers y nulos antes de clustering o modelos predictivos.

In [ ]:
# 1. Cargar el dataset original
df = pd.read_csv('../data/df_challenge_meli.csv')

# 2. Feature engineering inicial
df['discount_pct'] = np.where(
    df['regular_price'].isna() | (df['regular_price'] == 0),
    0,
    (df['regular_price'] - df['price']) / df['regular_price']
)

df['title_len'] = df['titulo'].astype(str).str.len()

rep_map = {
    "green_platinum": 5, "green_gold": 4, "green_silver": 4,
    "green": 3, "light_green": 3, "yellow": 2,
    "orange": 1, "red": 1, "newbie": 0
}
df['rep_score'] = df['seller_reputation'].map(rep_map).fillna(0)

# 3. Agregación a nivel seller
seller = (
    df.groupby('seller_nickname')
      .agg(
          num_publicaciones=('titulo', 'size'),
          log_stock_avg=('stock', lambda s: np.log1p(s).mean()),
          log_price_avg=('price', lambda s: np.log1p(s).mean()),
          porc_descuento=('discount_pct', 'mean'),
          categorias_distintas=('category_id', 'nunique'),
          proporcion_usados=('condition', lambda s: (s == 'used').mean()),
          proporcion_refurb=('is_refurbished', 'mean'), #validar el porque la imputacion de la media 
          rep_score=('rep_score', 'mean'),
          titulo_length_avg=('title_len', 'mean')
      )
      .reset_index()
)

# 4. Definir pipeline (imputación + escalado robusto)
num_cols = seller.columns.difference(['seller_nickname'])
X = seller[num_cols]

pre_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Aplicar las transformaciones
X_scaled = pre_pipe.fit_transform(X)

# 5. Persistir resultados en un DataFrame limpio
df_final = pd.DataFrame(X_scaled, columns=num_cols)
df_final.insert(0, 'seller_nickname', seller['seller_nickname'])

# Guardar dataset limpio y transformado
df_final.to_csv('../data/df_challenge_meli_limpio.csv', index=False)

# Confirmación visual
print(df_final.head())


  seller_nickname  categorias_distintas  log_price_avg  log_stock_avg  \
0      000631669c                   0.0       0.179097       0.199550   
1      0007153bca                   0.0      -0.251401       0.546386   
2      000bee3c3b                   0.0      -0.394075      -0.675326   
3      000df2bd02                   0.0       0.590278      -0.021600   
4      000e27cea2                   1.0      -0.208310      -0.181308   

   num_publicaciones  porc_descuento  proporcion_refurb  proporcion_usados  \
0                0.0             0.0                0.0                0.0   
1                0.5             0.0                0.0                0.0   
2                0.5             0.0                0.0                0.0   
3                0.0             0.0                0.0                1.0   
4                0.5             0.0                0.0                0.0   

   rep_score  titulo_length_avg  
0  -1.000000          -2.900763  
1   0.000000           0